In [1]:
import torch
from torch import nn
from torch import functional as F
from torch import optim

# 卷积

In [2]:
x = torch.rand(1, 3, 28, 28)
layer = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
out = layer(x)
print(layer.weight.shape)
print(layer.bias.shape)
print(out.shape)

torch.Size([8, 3, 3, 3])
torch.Size([8])
torch.Size([1, 8, 28, 28])



# pooling

In [3]:
pool_out=nn.MaxPool2d(2,stride=2)(out)
print(pool_out.shape)

torch.Size([1, 8, 14, 14])


In [4]:
out2 = torch.nn.functional.interpolate(out,scale_factor=2, mode='nearest')
print(out2.shape)

torch.Size([1, 8, 56, 56])


# RULE

In [5]:
layer = nn.ReLU()
out=layer(x)
print(out.shape)

torch.Size([1, 3, 28, 28])


# nn.Module

In [6]:
net = nn.Sequential(
    nn.Linear(10,5),
    nn.Linear(5,2)
)
x=torch.rand(1,3,10)
out=net(x)
print(out.shape)
print(list(net.parameters()))
print(dict(net.named_parameters()))

torch.Size([1, 3, 2])
[Parameter containing:
tensor([[ 0.1291, -0.2676,  0.0538, -0.1489, -0.0401, -0.0956, -0.0282, -0.0294,
         -0.2129, -0.0744],
        [-0.2307, -0.1350, -0.1951, -0.2076,  0.2921, -0.1934, -0.0930, -0.1362,
          0.0323,  0.0283],
        [-0.1185,  0.0069,  0.0678,  0.2101,  0.1073, -0.2848,  0.0418, -0.1806,
         -0.0574, -0.1409],
        [-0.1297,  0.1089, -0.0963, -0.2796, -0.2100, -0.0847,  0.1991, -0.0802,
         -0.2569,  0.2674],
        [-0.1555, -0.1580, -0.1851,  0.1474,  0.1742, -0.0226, -0.1863,  0.2197,
          0.1943,  0.1987]], requires_grad=True), Parameter containing:
tensor([ 0.2001, -0.0812,  0.2977,  0.1690, -0.2542], requires_grad=True), Parameter containing:
tensor([[ 0.4130, -0.3487,  0.2845, -0.3395,  0.4227],
        [ 0.0432, -0.3015,  0.2919,  0.1334,  0.1807]], requires_grad=True), Parameter containing:
tensor([-0.0959,  0.1810], requires_grad=True)]
{'0.weight': Parameter containing:
tensor([[ 0.1291, -0.2676,  0.05

In [10]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet,self).__init__()
        self.net = nn.Linear(4, 3)
    def forward(self, input):
        x = self.net(input)
        return x

class Mynet(nn.Module):
    def __init__(self):
        super(Mynet, self).__init__()
        self.mynet = nn.Sequential(
            BaseNet(),
            nn.ReLU(True),
            nn.Linear(3,2)
        )
    def forward(self, input):
        x = self.mynet(input)
        return x 
    
net = Mynet()
x = torch.rand(4,4)
logit = net(x)
print(logit.shape)
print(list(net.children()))
print(list(net.modules()))

torch.Size([4, 2])
[Sequential(
  (0): BaseNet(
    (net): Linear(in_features=4, out_features=3, bias=True)
  )
  (1): ReLU(inplace=True)
  (2): Linear(in_features=3, out_features=2, bias=True)
)]
[Mynet(
  (mynet): Sequential(
    (0): BaseNet(
      (net): Linear(in_features=4, out_features=3, bias=True)
    )
    (1): ReLU(inplace=True)
    (2): Linear(in_features=3, out_features=2, bias=True)
  )
), Sequential(
  (0): BaseNet(
    (net): Linear(in_features=4, out_features=3, bias=True)
  )
  (1): ReLU(inplace=True)
  (2): Linear(in_features=3, out_features=2, bias=True)
), BaseNet(
  (net): Linear(in_features=4, out_features=3, bias=True)
), Linear(in_features=4, out_features=3, bias=True), ReLU(inplace=True), Linear(in_features=3, out_features=2, bias=True)]


# load and save


In [13]:
net.load_state_dict(torch.load('ckpt.mdl'))

<All keys matched successfully>

In [12]:
torch.save(net.state_dict(),'ckpt.mdl')

# 实现自定义的层

In [15]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__(self)
        
    def forward(self, input):
        return x.view(input.shape[0],-1)


In [21]:
class MyLinear(nn.Module):
    def __init__(self):
        super(MyLinear, self).__init__()
        self.w = nn.Parameter(torch.rand(4,5))
        self.b = nn.Parameter(torch.rand(5))
        
    def forward(self, input):
        out = input@self.w.t() +self.b
        return out

layer = MyLinear()
list(layer.parameters())

[Parameter containing:
 tensor([[0.6175, 0.8351, 0.8340, 0.0837, 0.6190],
         [0.4812, 0.6389, 0.9192, 0.0786, 0.1406],
         [0.6982, 0.8834, 0.1657, 0.8518, 0.5994],
         [0.8682, 0.6035, 0.8661, 0.3129, 0.3741]], requires_grad=True),
 Parameter containing:
 tensor([0.5590, 0.8001, 0.9710, 0.9147, 0.1790], requires_grad=True)]